# NicheNet’s ligand activity analysis on a gene set of interest: predict  active ligands and their target genes 
__________________________________
Robin Browaeys 
2019-01-17 
 
<!-- github markdown built using  
rmarkdown::render("vignettes/ligand_activity_geneset.Rmd", output_format = "github_document") 
--> 
 
In this vignette, you can learn how to perform a basic NicheNet 
analysis. A NicheNet analysis can help you to generate hypotheses about 
an intercellular communication process of interest for which you have 
bulk or single-cell gene expression data. Specifically, NicheNet can 
predict 1) which ligands from one cell population (“sender/niche”) are 
most likely to affect target gene expression in an interacting cell 
population (“receiver/target”) and 2) which specific target genes are 
affected by which of these predicted ligands. 
 
Because NicheNet studies how ligands affect gene expression in 
neighboring cells, you need to have data about this effect in gene 
expression you want to study. So, you need to have a clear set of genes 
that are putatively affected by ligands from one of more interacting 
cells. 
 
The pipeline of a basic NicheNet analysis consist mainly of the 
following steps: 
 
  - 1.  Define a “sender/niche” cell population and a “receiver/target” 
        cell population present in your expression data and determine 
        which genes are expressed in both populations 
 
  - 2.  Define a gene set of interest: these are the genes in the 
        “receiver/target” cell population that are potentially 
        affected by ligands expressed by interacting cells (e.g. genes 
        differentially expressed upon cell-cell interaction) 
 
  - 3.  Define a set of potential ligands: these are ligands that are 
        expressed by the “sender/niche” cell population and bind a 
        (putative) receptor expressed by the “receiver/target” 
        population 
 
  - 4)  Perform NicheNet ligand activity analysis: rank the potential 
        ligands based on the presence of their target genes in the gene 
        set of interest (compared to the background set of genes) 
 
  - 5)  Infer top-predicted target genes of ligands that are top-ranked 
        in the ligand activity analysis 
 
This vignette guides you in detail through all these steps. As example 
expression data of interacting cells, we will use data from Puram et 
al. to explore intercellular communication in the tumor 
microenvironment in head and neck squamous cell carcinoma (HNSCC) (See 
Puram et al. 2017). More specifically, we will look at which ligands 
expressed by cancer-associated fibroblasts (CAFs) can induce a specific 
gene program in neighboring malignant cells. This program, a partial 
epithelial-mesenschymal transition (p-EMT) program, could be linked to 
metastasis by Puram et al.  
 
The used ligand-target matrix and example expression data of interacting 
cells can be downloaded from Zenodo. 
[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.3260758.svg)](https://doi.org/10.5281/zenodo.3260758) 
 

## Step 0: Load required packages, NicheNet’s ligand-target prior model and processed expression data of interacting cells 
 
Packages: 
 

In [1]:
library(nichenetr) 
library(tidyverse)


Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   1.0.1 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.5.0 
✔ readr   2.1.3      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


 
Ligand-target model: 
 
This model denotes the prior potential that a particular ligand might 
regulate the expression of a specific target gene. 
 


In [2]:
ligand_target_matrix = readRDS(url("https://zenodo.org/record/3260758/files/ligand_target_matrix.rds")) 
ligand_target_matrix[1:5,1:5] # target genes in rows, ligands in columns 
##                 CXCL1        CXCL2        CXCL3        CXCL5         PPBP 
## A1BG     3.534343e-04 4.041324e-04 3.729920e-04 3.080640e-04 2.628388e-04 
## A1BG-AS1 1.650894e-04 1.509213e-04 1.583594e-04 1.317253e-04 1.231819e-04 
## A1CF     5.787175e-04 4.596295e-04 3.895907e-04 3.293275e-04 3.211944e-04 
## A2M      6.027058e-04 5.996617e-04 5.164365e-04 4.517236e-04 4.590521e-04 
## A2M-AS1  8.898724e-05 8.243341e-05 7.484018e-05 4.912514e-05 5.120439e-05 

,CXCL1,CXCL2,CXCL3,CXCL5,PPBP
A1BG,3.534343e-04,4.041324e-04,3.729920e-04,3.080640e-04,2.628388e-04
A1BG-AS1,1.650894e-04,1.509213e-04,1.583594e-04,1.317253e-04,1.231819e-04
A1CF,5.787175e-04,4.596295e-04,3.895907e-04,3.293275e-04,3.211944e-04
A2M,6.027058e-04,5.996617e-04,5.164365e-04,4.517236e-04,4.590521e-04
A2M-AS1,8.898724e-05,8.243341e-05,7.484018e-05,4.912514e-05,5.120439e-05


 
Expression data of interacting cells: publicly available single-cell 
data from CAF and malignant cells from HNSCC tumors: 
 


In [2]:
hnscc_expression = readRDS(file.path("~/FCA_Fetal_Skin_priv/data/hnscc_expression.rds")) 
expression = hnscc_expression$expression 
sample_info = hnscc_expression$sample_info # contains meta-information about the cells 


In [10]:
rownames(sample_info)

[1] "1"    "2"    "3"    "4"    "5"    "6"    "7"    "8"    "9"    "10"  
  [11] "11"   "12"   "13"   "14"   "15"   "16"   "17"   "18"   "19"   "20"  
  [21] "21"   "22"   "23"   "24"   "25"   "26"   "27"   "28"   "29"   "30"  
  [31] "31"   "32"   "33"   "34"   "35"   "36"   "37"   "38"   "39"   "40"  
  [41] "41"   "42"   "43"   "44"   "45"   "46"   "47"   "48"   "49"   "50"  
  [51] "51"   "52"   "53"   "54"   "55"   "56"   "57"   "58"   "59"   "60"  
  [61] "61"   "62"   "63"   "64"   "65"   "66"   "67"   "68"   "69"   "70"  
  [71] "71"   "72"   "73"   "74"   "75"   "76"   "77"   "78"   "79"   "80"  
  [81] "81"   "82"   "83"   "84"   "85"   "86"   "87"   "88"   "89"   "90"  
  [91] "91"   "92"   "93"   "94"   "95"   "96"   "97"   "98"   "99"   "100" 
 [101] "101"  "102"  "103"  "104"  "105"  "106"  "107"  "108"  "109"  "110" 
 [111] "111"  "112"  "113"  "114"  "115"  "116"  "117"  "118"  "119"  "120" 
 [121] "121"  "122"  "123"  "124"  "125"  "126"  "127"  "128"  "129"  "130" 
 [131] "131"  "132"  "133"  "134"  "135"  "136"  "137"  "138"  "139"  "140" 
 [141] "141"  "142"  "143"  "144"  "145"  "146"  "147"  "148"  "149"  "150" 
 [151] "151"  "152"  "153"  "154"  "155"  "156"  "157"  "158"  "159"  "160" 
 [161] "161"  "162"  "163"  "164"  "165"  "166"  "167"  "168"  "169"  "170" 
 [171] "171"  "172"  "173"  "174"  "175"  "176"  "177"  "178"  "179"  "180" 
 [181] "181"  "182"  "183"  "184"  "185"  "186"  "187"  "188"  "189"  "190" 
 [191] "191"  "192"  "193"  "194"  "195"  "196"  "197"  "198"  "199"  "200" 
 [201] "201"  "202"  "203"  "204"  "205"  "206"  "207"  "208"  "209"  "210" 
 [211] "211"  "212"  "213"  "214"  "215"  "216"  "217"  "218"  "219"  "220" 
 [221] "221"  "222"  "223"  "224"  "225"  "226"  "227"  "228"  "229"  "230" 
 [231] "231"  "232"  "233"  "234"  "235"  "236"  "237"  "238"  "239"  "240" 
 [241] "241"  "242"  "243"  "244"  "245"  "246"  "247"  "248"  "249"  "250" 
 [251] "251"  "252"  "253"  "254"  "255"  "256"  "257"  "258"  "259"  "260" 
 [261] "261"  "262"  "263"  "264"  "265"  "266"  "267"  "268"  "269"  "270" 
 [271] "271"  "272"  "273"  "274"  "275"  "276"  "277"  "278"  "279"  "280" 
 [281] "281"  "282"  "283"  "284"  "285"  "286"  "287"  "288"  "289"  "290" 
 [291] "291"  "292"  "293"  "294"  "295"  "296"  "297"  "298"  "299"  "300" 
 [301] "301"  "302"  "303"  "304"  "305"  "306"  "307"  "308"  "309"  "310" 
 [311] "311"  "312"  "313"  "314"  "315"  "316"  "317"  "318"  "319"  "320" 
 [321] "321"  "322"  "323"  "324"  "325"  "326"  "327"  "328"  "329"  "330" 
 [331] "331"  "332"  "333"  "334"  "335"  "336"  "337"  "338"  "339"  "340" 
 [341] "341"  "342"  "343"  "344"  "345"  "346"  "347"  "348"  "349"  "350" 
 [351] "351"  "352"  "353"  "354"  "355"  "356"  "357"  "358"  "359"  "360" 
 [361] "361"  "362"  "363"  "364"  "365"  "366"  "367"  "368"  "369"  "370" 
 [371] "371"  "372"  "373"  "374"  "375"  "376"  "377"  "378"  "379"  "380" 
 [381] "381"  "382"  "383"  "384"  "385"  "386"  "387"  "388"  "389"  "390" 
 [391] "391"  "392"  "393"  "394"  "395"  "396"  "397"  "398"  "399"  "400" 
 [401] "401"  "402"  "403"  "404"  "405"  "406"  "407"  "408"  "409"  "410" 
 [411] "411"  "412"  "413"  "414"  "415"  "416"  "417"  "418"  "419"  "420" 
 [421] "421"  "422"  "423"  "424"  "425"  "426"  "427"  "428"  "429"  "430" 
 [431] "431"  "432"  "433"  "434"  "435"  "436"  "437"  "438"  "439"  "440" 
 [441] "441"  "442"  "443"  "444"  "445"  "446"  "447"  "448"  "449"  "450" 
 [451] "451"  "452"  "453"  "454"  "455"  "456"  "457"  "458"  "459"  "460" 
 [461] "461"  "462"  "463"  "464"  "465"  "466"  "467"  "468"  "469"  "470" 
 [471] "471"  "472"  "473"  "474"  "475"  "476"  "477"  "478"  "479"  "480" 
 [481] "481"  "482"  "483"  "484"  "485"  "486"  "487"  "488"  "489"  "490" 
 [491] "491"  "492"  "493"  "494"  "495"  "496"  "497"  "498"  "499"  "500" 
 [501] "501"  "502"  "503"  "504"  "505"  "506"  "507"  "508"  "509"  "510" 
 [511] "511"  "512"  "513"  "514"  "515"  "516"  "517"  "518"  "519"  "520" 

 
## Step 1: Define expressed genes in sender and receiver cell populations 
 
Our research question is to prioritize which ligands expressed by CAFs 
can induce p-EMT in neighboring malignant cells. Therefore, CAFs are the 
sender cells in this example and malignant cells are the receiver cells. 
This is an example of paracrine signaling. Note that autocrine signaling 
can be considered if sender and receiver cell type are the same. 
 
Now, we will determine which genes are expressed in the sender cells 
(CAFs) and receiver cells (malignant cells) from high quality primary 
tumors. Therefore, we wil not consider cells from tumor samples of less 
quality or from lymph node metastases. 
 
To determine expressed genes in this case study, we use the definition 
used by Puram et al. (the authors of this dataset), which is: Ea, the 
aggregate expression of each gene i across the k cells, calculated as 
Ea(i) = log2(average(TPM(i)1…k)+1), should be \>= 4. We recommend users 
to define expressed genes in the way that they consider to be most 
appropriate for their dataset. For single-cell data generated by the 10x 
platform in our lab, we don’t use the definition used here, but we 
consider genes to be expressed in a cell type when they have non-zero 
values in at least 10% of the cells from that cell type. This is 
described as well in the other vignette [Perform NicheNet analysis 
starting from a Seurat object: step-by-step 
analysis](seurat_steps.md):`vignette("seurat_steps", 
package="nichenetr")`. 
 


In [4]:
tumors_remove = c("HN10","HN","HN12", "HN13", "HN24", "HN7", "HN8","HN23") 
 
CAF_ids = sample_info %>% filter(`Lymph node` == 0 & !(tumor %in% tumors_remove) & `non-cancer cell type` == "CAF") %>% pull(cell) 
malignant_ids = sample_info %>% filter(`Lymph node` == 0 & !(tumor %in% tumors_remove) & `classified  as cancer cell` == 1) %>% pull(cell) 

In [5]:
CAF_ids

[1] "HN25_P5_C05_S29_comb"            "HN26_P6_A08_S8_comb"            
  [3] "HN28_P6_A02_S98_comb"            "HN28_P6_E12_S156_comb"          
  [5] "HN26_P6_G03_S75_comb"            "HN26_P5_G03_S75_comb"           
  [7] "HN28_P6_E09_S153_comb"           "HN28_P6_D12_S144_comb"          
  [9] "HN28_P6_B07_S115_comb"           "HN25_P5_E05_S53_comb"           
 [11] "HN28_P6_F05_S161_comb"           "HN28_P6_D08_S140_comb"          
 [13] "HN28_P6_G08_S176_comb"           "HN28_P6_F12_S168_comb"          
 [15] "HN26_P6_F07_S67_comb"            "HN28_P6_C02_S122_comb"          
 [17] "HN28_P6_D06_S138_comb"           "HN25_P5_D07_S43_comb"           
 [19] "HN25_P5_C12_S36_comb"            "HN25_P5_C08_S32_comb"           
 [21] "HN25_P5_E08_S56_comb"            "HN25_P5_A12_S12_comb"           
 [23] "HN28_P6_A03_S99_comb"            "HN28_P6_F01_S157_comb"          
 [25] "HN28_P6_B12_S120_comb"           "HN25_P5_E04_S52_comb"           
 [27] "HN28_P6_C05_S125_comb"           "HN25_P5_A08_S8_comb"            
 [29] "HN28_P6_F03_S159_comb"           "HN26_P6_E02_S50_comb"           
 [31] "HN28_P6_A05_S101_comb"           "HN28_P6_A04_S100_comb"          
 [33] "HN25_P5_A10_S10_comb"            "HN28_P6_B06_S114_comb"          
 [35] "HN28_P6_E04_S148_comb"           "HN25_P5_H01_S85_comb"           
 [37] "HN26_P5_F12_S72_comb"            "HN26_P6_F09_S69_comb"           
 [39] "HN28_P6_C10_S130_comb"           "HN28_P6_D04_S136_comb"          
 [41] "HN25_P5_D02_S38_comb"            "HN28_P6_F06_S162_comb"          
 [43] "HN28_P5_D11_S239_comb"           "HN28_P3P4_A12_S12_comb"         
 [45] "HN28_P5_G10_S274_comb"           "HN28_P5_G12_S276_comb"          
 [47] "HN28_P5_F08_S260_comb"           "HN28_P5_F02_S254_comb"          
 [49] "HN28_P3P4_C11_S35_comb"          "HN28_P3P4_C04_S28_comb"         
 [51] "HN28_P5_H11_S287_comb"           "HN28_P3P4_B02_S14_comb"         
 [53] "HN28_P3P4_E03_S51_comb"          "HN28_P3P4_B06_S18_comb"         
 [55] "HN28_P5_D07_S235_comb"           "HN28_P3P4_G02_S74_comb"         
 [57] "HN28_P3P4_F05_S65_comb"          "HN28_P5_F06_S258_comb"          
 [59] "HN28_P5_D02_S230_comb"           "HN28_P5_G01_S265_comb"          
 [61] "HN28_P3P4_G04_S76_comb"          "HN28_P3P4_A01_S1_comb"          
 [63] "HN28_P3P4_H05_S89_comb"          "HN28_P5_E10_S250_comb"          
 [65] "HN28_P3P4_H09_S93_comb"          "HN28_P3P4_B01_S13_comb"         
 [67] "HN28_P5_H03_S279_comb"           "HN28_P3P4_E07_S55_comb"         
 [69] "HN28_P5_C10_S226_comb"           "HN28_P5_B11_S215_comb"          
 [71] "HN28_P5_B10_S214_comb"           "HN28_P3P4_G01_S73_comb"         
 [73] "HN28_P5_F04_S256_comb"           "HN28_P3P4_B07_S19_comb"         
 [75] "HN28_P5_C07_S223_comb"           "HN28_P5_A01_S193_comb"          
 [77] "HN28_P3P4_F04_S64_comb"          "HN28_P3P4_A05_S5_comb"          
 [79] "HN28_P5_F07_S259_comb"           "HN28_P3P4_F09_S69_comb"         
 [81] "HN28_P3P4_A11_S11_comb"          "HN28_P3P4_F01_S61_comb"         
 [83] "HN28_P3P4_B05_S17_comb"          "HN28_P5_A08_S200_comb"          
 [85] "HN28_P5_H05_S281_comb"           "HN28_P3P4_G07_S79_comb"         
 [87] "HN28_P3P4_G08_S80_comb"          "HN28_P3P4_H03_S87_comb"         
 [89] "HN28_P3P4_A10_S10_comb"          "HN28_P3P4_C03_S27_comb"         
 [91] "HN28_P3P4_D03_S39_comb"          "HN28_P3P4_C05_S29_comb"         
 [93] "HNSCC25_P2_H08_S92_comb"         "HNSCC25_P2_A04_S4_comb"         
 [95] "HNSCC25_P2_B11_S23_comb"         "HNSCC25_P2_C09_S33_comb"        
 [97] "HNSCC25_P2_G12_S84_comb"         "HNSCC25_P2_D09_S45_comb"        
 [99] "HNSCC25_P2_E03_S51_comb"         "HNSCC25_P2_B03_S15_comb"        
[101] "HNSCC25_P2_D06_S42_comb"         "HNSCC25_P2_C11_S35_comb"        
[103] "HNSCC25_P2_B04_S16_comb"         "HNSCC25_P2_D02_S38_comb"        
[105] "HNSCC26_P3_G12_S180_comb"        "HNSCC26_P3_D10_S142_comb"       
[107] "HNSCC26_P3_H10_S190_comb"        "HNSCC26_P3_E12_S156_comb"       
[109] 

In [6]:
 
expressed_genes_sender = expression[CAF_ids,] %>% apply(2,function(x){10*(2**x - 1)}) %>% apply(2,function(x){log2(mean(x) + 1)}) %>% .[. >= 4] %>% names() 
expressed_genes_receiver = expression[malignant_ids,] %>% apply(2,function(x){10*(2**x - 1)}) %>% apply(2,function(x){log2(mean(x) + 1)}) %>% .[. >= 4] %>% names() 
 
# Check the number of expressed genes: should be a 'reasonable' number of total expressed genes in a cell type, e.g. between 5000-10000 (and not 500 or 20000) 
length(expressed_genes_sender) 
## [1] 6706 
length(expressed_genes_receiver) 
## [1] 6351 

[1] 6706

[1] 6351

In [8]:
unique(expression[CAF_ids,])

,C9orf152,RPS11,ELMO2,CREB3L1,PNMA1,MMP2,TMEM216,TRAF3IP2-AS1,LRRC37A5P,LOC653712,⋯,GPLD1,SNORD115-39,RAB8A,RXFP2,PCIF1,PIK3IP1,SNRPD2,SLC39A6,CTSC,AQP7
HN25_P5_C05_S29_comb,0,5.2684,3.3842,0.0000,0.0000,6.8067,0.0000,0.16736,0,0.00000,⋯,0.000000,0,2.4306,0,0.00000,0.0000,4.0511,0.00000,6.093000,0
HN26_P6_A08_S8_comb,0,7.5317,0.0000,0.0000,0.0000,9.0488,6.4585,0.17887,0,0.00000,⋯,0.186500,0,0.0000,0,0.00000,0.0000,0.0000,0.00000,0.662660,0
HN28_P6_A02_S98_comb,0,6.8341,0.0000,0.0000,5.9557,6.5373,6.0280,0.34937,0,0.00000,⋯,0.000000,0,0.0000,0,0.00000,0.0000,4.7276,0.00000,0.498760,0
HN28_P6_E12_S156_comb,0,5.7698,5.7006,0.0000,5.1591,5.4166,0.0000,0.75617,0,0.00000,⋯,0.280960,0,4.3308,0,0.00000,4.9251,4.7076,0.55876,0.298660,0
HN26_P6_G03_S75_comb,0,7.3069,0.0000,0.0000,3.9981,8.4103,5.0065,0.37962,0,0.00000,⋯,0.000000,0,0.0000,0,0.00000,0.0000,4.9188,0.00000,0.079975,0
HN26_P5_G03_S75_comb,0,5.2336,0.0000,0.0000,0.0000,4.9614,5.3029,0.28570,0,0.00000,⋯,0.176320,0,0.0000,0,0.00000,0.0000,0.0000,4.69130,2.583800,0
HN28_P6_E09_S153_comb,0,5.9016,0.0000,0.0000,0.0000,7.3070,0.0000,0.31150,0,0.00000,⋯,0.000000,0,0.0000,0,0.00000,0.0000,4.5981,0.00000,0.786600,0
HN28_P6_D12_S144_comb,0,7.2601,0.0000,0.0000,0.0000,4.9869,0.0000,0.32077,0,0.00000,⋯,0.000000,0,0.0000,0,1.75700,0.0000,4.3394,4.86200,0.413590,0
HN28_P6_B07_S115_comb,0,4.7733,0.0000,0.0000,0.0000,6.8850,5.7087,0.00000,0,0.00000,⋯,2.458400,0,0.0000,0,0.00000,4.6038,0.0000,3.28200,0.504870,0
HN25_P5_E05_S53_comb,0,6.6916,0.0000,0.0000,3.8469,8.6583,3.7982,0.10836,0,0.00000,⋯,0.093560,0,3.1440,0,3.41590,0.0000,4.7124,3.28320,3.973300,0


 
## Step 2: Define the gene set of interest and a background of genes 
 
As gene set of interest, we consider the genes of which the expression 
is possibly affected due to communication with other cells. The 
definition of this gene set depends on your research question and is a 
crucial step in the use of NicheNet. 
 
Because we here want to investigate how CAFs regulate the expression of 
p-EMT genes in malignant cells, we will use the p-EMT gene set defined 
by Puram et al. as gene set of interest and use all genes expressed in 
malignant cells as background of genes. 
 


In [ ]:
geneset_oi = readr::read_tsv(url("https://zenodo.org/record/3260758/files/pemt_signature.txt"), col_names = "gene") %>% pull(gene) %>% .[. %in% rownames(ligand_target_matrix)] # only consider genes also present in the NicheNet model - this excludes genes from the gene list for which the official HGNC symbol was not used by Puram et al. 
head(geneset_oi) 
## [1] "SERPINE1" "TGFBI"    "MMP10"    "LAMC2"    "P4HA2"    "PDPN" 
 
background_expressed_genes = expressed_genes_receiver %>% .[. %in% rownames(ligand_target_matrix)] 
head(background_expressed_genes) 
## [1] "RPS11"   "ELMO2"   "PNMA1"   "MMP2"    "TMEM216" "ERCC5" 


 
## Step 3: Define a set of potential ligands 
 
As potentially active ligands, we will use ligands that are 1) expressed 
by CAFs and 2) can bind a (putative) receptor expressed by malignant 
cells. Putative ligand-receptor links were gathered from NicheNet’s 
ligand-receptor data sources. 
 


In [ ]:
lr_network = readRDS(url("https://zenodo.org/record/3260758/files/lr_network.rds")) 
 
# If wanted, users can remove ligand-receptor interactions that were predicted based on protein-protein interactions and only keep ligand-receptor interactions that are described in curated databases. To do this: uncomment following line of code: 
# lr_network = lr_network %>% filter(database != "ppi_prediction_go" & database != "ppi_prediction") 
 
ligands = lr_network %>% pull(from) %>% unique() 
expressed_ligands = intersect(ligands,expressed_genes_sender) 
 
receptors = lr_network %>% pull(to) %>% unique() 
expressed_receptors = intersect(receptors,expressed_genes_receiver) 
 
lr_network_expressed = lr_network %>% filter(from %in% expressed_ligands & to %in% expressed_receptors)  
head(lr_network_expressed) 
## # A tibble: 6 x 4 
##   from    to        source         database 
##   <chr>   <chr>     <chr>          <chr>    
## 1 HGF     MET       kegg_cytokines kegg     
## 2 TNFSF10 TNFRSF10A kegg_cytokines kegg     
## 3 TNFSF10 TNFRSF10B kegg_cytokines kegg     
## 4 TGFB2   TGFBR1    kegg_cytokines kegg     
## 5 TGFB3   TGFBR1    kegg_cytokines kegg     
## 6 INHBA   ACVR2A    kegg_cytokines kegg 


 
This ligand-receptor network contains the expressed ligand-receptor 
interactions. As potentially active ligands for the NicheNet analysis, 
we will consider the ligands from this network. 
 


In [ ]:
potential_ligands = lr_network_expressed %>% pull(from) %>% unique() 
head(potential_ligands) 
## [1] "HGF"     "TNFSF10" "TGFB2"   "TGFB3"   "INHBA"   "CD99" 


 
## Step 4: Perform NicheNet’s ligand activity analysis on the gene set of interest 
 
Now perform the ligand activity analysis: in this analysis, we will 
calculate the ligand activity of each ligand, or in other words, we will 
assess how well each CAF-ligand can predict the p-EMT gene set compared 
to the background of expressed genes (predict whether a gene belongs to 
the p-EMT program or not). 
 


In [ ]:
ligand_activities = predict_ligand_activities(geneset = geneset_oi, background_expressed_genes = background_expressed_genes, ligand_target_matrix = ligand_target_matrix, potential_ligands = potential_ligands) 



 
Now, we want to rank the ligands based on their ligand activity. In our 
validation study, we showed that the pearson correlation coefficient 
(PCC) between a ligand’s target predictions and the observed 
transcriptional response was the most informative measure to define 
ligand activity. Therefore, we will rank the ligands based on their 
pearson correlation coefficient. This allows us to prioritize 
p-EMT-regulating ligands. 
 


In [ ]:
ligand_activities %>% arrange(-pearson)  
## # A tibble: 131 x 4 
##    test_ligand auroc   aupr pearson 
##    <chr>       <dbl>  <dbl>   <dbl> 
##  1 PTHLH       0.667 0.0720   0.128 
##  2 CXCL12      0.680 0.0507   0.123 
##  3 AGT         0.676 0.0581   0.120 
##  4 TGFB3       0.689 0.0454   0.117 
##  5 IL6         0.693 0.0510   0.115 
##  6 INHBA       0.695 0.0502   0.113 
##  7 ADAM17      0.672 0.0526   0.113 
##  8 TNC         0.700 0.0444   0.109 
##  9 CTGF        0.680 0.0473   0.108 
## 10 FN1         0.679 0.0505   0.108 
## # ... with 121 more rows 
best_upstream_ligands = ligand_activities %>% top_n(20, pearson) %>% arrange(-pearson) %>% pull(test_ligand) 
head(best_upstream_ligands) 
## [1] "PTHLH"  "CXCL12" "AGT"    "TGFB3"  "IL6"    "INHBA" 


 
We see here that the performance metrics indicate that the 20 top-ranked 
ligands can predict the p-EMT genes reasonably, this implies that 
ranking of the ligands might be accurate as shown in our study. However, 
it is possible that for some gene sets, the target gene prediction 
performance of the top-ranked ligands would not be much better than 
random prediction. In that case, prioritization of ligands will be less 
trustworthy. 
 
Additional note: we looked at the top 20 ligands here and will continue 
the analysis by inferring p-EMT target genes of these 20 ligands. 
However, the choice of looking only at the 20 top-ranked ligands for 
further biological interpretation is based on biological intuition and 
is quite arbitrary. Therefore, users can decide to continue the analysis 
with a different number of ligands. We recommend to check the selected 
cutoff by looking at the distribution of the ligand activity values. 
Here, we show the ligand activity histogram (the score for the 20th 
ligand is indicated via the dashed line). 
 


In [ ]:
# show histogram of ligand activity scores 
p_hist_lig_activity = ggplot(ligand_activities, aes(x=pearson)) +  
  geom_histogram(color="black", fill="darkorange")  +  
  # geom_density(alpha=.1, fill="orange") + 
  geom_vline(aes(xintercept=min(ligand_activities %>% top_n(20, pearson) %>% pull(pearson))), color="red", linetype="dashed", size=1) +  
  labs(x="ligand activity (PCC)", y = "# ligands") + 
  theme_classic() 
p_hist_lig_activity 


 
![](ligand_activity_geneset_files/figure-gfm/unnamed-chunk-10-1.png)<!-- --> 
 
## Step 5: Infer target genes of top-ranked ligands and visualize in a heatmap 
 
Now we will show how you can look at the regulatory potential scores 
between ligands and target genes of interest. In this case, we will look 
at links between top-ranked p-EMT regulating ligands and p-EMT genes. In 
the ligand-target heatmaps, we show here regulatory potential scores for 
interactions between the 20 top-ranked ligands and following target 
genes: genes that belong to the gene set of interest and to the 250 most 
strongly predicted targets of at least one of the 20 top-ranked ligands 
(the top 250 targets according to the general prior model, so not the 
top 250 targets for this dataset). Consequently, genes of your gene set 
that are not a top target gene of one of the prioritized ligands, will 
not be shown on the heatmap. 
 


In [ ]:
active_ligand_target_links_df = best_upstream_ligands %>% lapply(get_weighted_ligand_target_links,geneset = geneset_oi, ligand_target_matrix = ligand_target_matrix, n = 250) %>% bind_rows() 
 
nrow(active_ligand_target_links_df) 
## [1] 143 
head(active_ligand_target_links_df) 
## # A tibble: 6 x 3 
##   ligand target  weight 
##   <chr>  <chr>    <dbl> 
## 1 PTHLH  COL1A1 0.00399 
## 2 PTHLH  MMP1   0.00425 
## 3 PTHLH  MMP2   0.00210 
## 4 PTHLH  MYH9   0.00116 
## 5 PTHLH  P4HA2  0.00190 
## 6 PTHLH  PLAU   0.00401 


 
For visualization purposes, we adapted the ligand-target regulatory 
potential matrix as follows. Regulatory potential scores were set as 0 
if their score was below a predefined threshold, which was here the 0.25 
quantile of scores of interactions between the 20 top-ranked ligands and 
each of their respective top targets (see the ligand-target network 
defined in the data frame). 
 


In [ ]:
active_ligand_target_links = prepare_ligand_target_visualization(ligand_target_df = active_ligand_target_links_df, ligand_target_matrix = ligand_target_matrix, cutoff = 0.25) 
 
nrow(active_ligand_target_links_df) 
## [1] 143 
head(active_ligand_target_links_df) 
## # A tibble: 6 x 3 
##   ligand target  weight 
##   <chr>  <chr>    <dbl> 
## 1 PTHLH  COL1A1 0.00399 
## 2 PTHLH  MMP1   0.00425 
## 3 PTHLH  MMP2   0.00210 
## 4 PTHLH  MYH9   0.00116 
## 5 PTHLH  P4HA2  0.00190 
## 6 PTHLH  PLAU   0.00401 


 
The putatively active ligand-target links will now be visualized in a 
heatmap. The order of the ligands accord to the ranking according to the 
ligand activity prediction. 
 


In [ ]:
order_ligands = intersect(best_upstream_ligands, colnames(active_ligand_target_links)) %>% rev() 
order_targets = active_ligand_target_links_df$target %>% unique() 
vis_ligand_target = active_ligand_target_links[order_targets,order_ligands] %>% t() 
 
p_ligand_target_network = vis_ligand_target %>% make_heatmap_ggplot("Prioritized CAF-ligands","p-EMT genes in malignant cells", color = "purple",legend_position = "top", x_axis_position = "top",legend_title = "Regulatory potential") + scale_fill_gradient2(low = "whitesmoke",  high = "purple", breaks = c(0,0.005,0.01)) + theme(axis.text.x = element_text(face = "italic")) 
 
p_ligand_target_network 


 
![](ligand_activity_geneset_files/figure-gfm/unnamed-chunk-13-1.png)<!-- --> 
 
Note that the choice of these cutoffs for visualization is quite 
arbitrary. We recommend users to test several cutoff values. 
 
If you would consider more than the top 250 targets based on prior 
information, you will infer more, but less confident, ligand-target 
links; by considering less than 250 targets, you will be more stringent. 
 
If you would change the quantile cutoff that is used to set scores to 0 
(for visualization purposes), lowering this cutoff will result in a more 
dense heatmap, whereas highering this cutoff will result in a more 
sparse heatmap. 
 
## Follow-up analysis 1: Ligand-receptor network inference for top-ranked ligands 
 
One type of follow-up analysis is looking at which receptors of the 
receiver cell population (here: malignant cells) can potentially bind to 
the prioritized ligands from the sender cell population (here: CAFs). 
 
So, we will now infer the predicted ligand-receptor interactions of the 
top-ranked ligands and visualize these in a heatmap. 
 


In [ ]:
# get the ligand-receptor network of the top-ranked ligands 
lr_network_top = lr_network %>% filter(from %in% best_upstream_ligands & to %in% expressed_receptors) %>% distinct(from,to) 
best_upstream_receptors = lr_network_top %>% pull(to) %>% unique() 
 
# get the weights of the ligand-receptor interactions as used in the NicheNet model 
weighted_networks = readRDS(url("https://zenodo.org/record/3260758/files/weighted_networks.rds")) 
lr_network_top_df = weighted_networks$lr_sig %>% filter(from %in% best_upstream_ligands & to %in% best_upstream_receptors) 
 
# convert to a matrix 
lr_network_top_df = lr_network_top_df %>% spread("from","weight",fill = 0) 
lr_network_top_matrix = lr_network_top_df %>% select(-to) %>% as.matrix() %>% magrittr::set_rownames(lr_network_top_df$to) 
 
# perform hierarchical clustering to order the ligands and receptors 
dist_receptors = dist(lr_network_top_matrix, method = "binary") 
hclust_receptors = hclust(dist_receptors, method = "ward.D2") 
order_receptors = hclust_receptors$labels[hclust_receptors$order] 
 
dist_ligands = dist(lr_network_top_matrix %>% t(), method = "binary") 
hclust_ligands = hclust(dist_ligands, method = "ward.D2") 
order_ligands_receptor = hclust_ligands$labels[hclust_ligands$order] 


 
Show a heatmap of the ligand-receptor interactions 
 


In [ ]:
vis_ligand_receptor_network = lr_network_top_matrix[order_receptors, order_ligands_receptor] 
p_ligand_receptor_network = vis_ligand_receptor_network %>% t() %>% make_heatmap_ggplot("Prioritized CAF-ligands","Receptors expressed by malignant cells", color = "mediumvioletred", x_axis_position = "top",legend_title = "Prior interaction potential") 
p_ligand_receptor_network 


 
![](ligand_activity_geneset_files/figure-gfm/unnamed-chunk-15-1.png)<!-- --> 
 
## Follow-up analysis 2: Visualize expression of top-predicted ligands and their target genes in a combined heatmap 
 
NicheNet only considers expressed ligands of sender cells, but does not 
take into account their expression for ranking the ligands. The ranking 
is purely based on the potential that a ligand might regulate the gene 
set of interest, given prior knowledge. Because it is also useful to 
further look into expression of ligands and their target genes, we 
demonstrate here how you could make a combined figure showing ligand 
activity, ligand expression, target gene expression and ligand-target 
regulatory potential. 
 
#### Load additional packages required for the visualization: 
 


In [ ]:
library(RColorBrewer) 
library(cowplot) 
library(ggpubr) 


 
#### Prepare the ligand activity matrix 
 


In [ ]:
ligand_pearson_matrix = ligand_activities %>% select(pearson) %>% as.matrix() %>% magrittr::set_rownames(ligand_activities$test_ligand) 
 
vis_ligand_pearson = ligand_pearson_matrix[order_ligands, ] %>% as.matrix(ncol = 1) %>% magrittr::set_colnames("Pearson") 


In [ ]:
p_ligand_pearson = vis_ligand_pearson %>% make_heatmap_ggplot("Prioritized CAF-ligands","Ligand activity", color = "darkorange",legend_position = "top", x_axis_position = "top", legend_title = "Pearson correlation coefficient\ntarget gene prediction ability)") 
p_ligand_pearson 


 
![](ligand_activity_geneset_files/figure-gfm/unnamed-chunk-18-1.png)<!-- --> 
 
#### Prepare expression of ligands in fibroblast per tumor 
 
Because the single-cell data was collected from multiple tumors, we will 
show here the average expression of the ligands per tumor. 
 


In [ ]:
expression_df_CAF = expression[CAF_ids,order_ligands] %>% data.frame() %>% rownames_to_column("cell") %>% as_tibble() %>% inner_join(sample_info %>% select(cell,tumor), by =  "cell") 
 
aggregated_expression_CAF = expression_df_CAF %>% group_by(tumor) %>% select(-cell) %>% summarise_all(mean) 
 
aggregated_expression_df_CAF = aggregated_expression_CAF %>% select(-tumor) %>% t() %>% magrittr::set_colnames(aggregated_expression_CAF$tumor) %>% data.frame() %>% rownames_to_column("ligand") %>% as_tibble()  
 
aggregated_expression_matrix_CAF = aggregated_expression_df_CAF %>% select(-ligand) %>% as.matrix() %>% magrittr::set_rownames(aggregated_expression_df_CAF$ligand) 
 
order_tumors = c("HN6","HN20","HN26","HN28","HN22","HN25","HN5","HN18","HN17","HN16") # this order was determined based on the paper from Puram et al. Tumors are ordered according to p-EMT score. 
vis_ligand_tumor_expression = aggregated_expression_matrix_CAF[order_ligands,order_tumors] 


In [ ]:
library(RColorBrewer) 
color = colorRampPalette(rev(brewer.pal(n = 7, name ="RdYlBu")))(100) 
p_ligand_tumor_expression = vis_ligand_tumor_expression %>% make_heatmap_ggplot("Prioritized CAF-ligands","Tumor", color = color[100],legend_position = "top", x_axis_position = "top", legend_title = "Expression\n(averaged over\nsingle cells)") + theme(axis.text.y = element_text(face = "italic")) 
p_ligand_tumor_expression 


 
![](ligand_activity_geneset_files/figure-gfm/unnamed-chunk-20-1.png)<!-- --> 
 
#### Prepare expression of target genes in malignant cells per tumor 
 


In [ ]:
expression_df_target = expression[malignant_ids,geneset_oi] %>% data.frame() %>% rownames_to_column("cell") %>% as_tibble() %>% inner_join(sample_info %>% select(cell,tumor), by =  "cell")  
 
aggregated_expression_target = expression_df_target %>% group_by(tumor) %>% select(-cell) %>% summarise_all(mean) 
 
aggregated_expression_df_target = aggregated_expression_target %>% select(-tumor) %>% t() %>% magrittr::set_colnames(aggregated_expression_target$tumor) %>% data.frame() %>% rownames_to_column("target") %>% as_tibble()  
 
aggregated_expression_matrix_target = aggregated_expression_df_target %>% select(-target) %>% as.matrix() %>% magrittr::set_rownames(aggregated_expression_df_target$target) 
 
vis_target_tumor_expression_scaled = aggregated_expression_matrix_target %>% t() %>% scale_quantile() %>% .[order_tumors,order_targets] 


In [ ]:
p_target_tumor_scaled_expression = vis_target_tumor_expression_scaled  %>% make_threecolor_heatmap_ggplot("Tumor","Target", low_color = color[1],mid_color = color[50], mid = 0.5, high_color = color[100], legend_position = "top", x_axis_position = "top" , legend_title = "Scaled expression\n(averaged over\nsingle cells)") + theme(axis.text.x = element_text(face = "italic")) 
p_target_tumor_scaled_expression 


 
![](ligand_activity_geneset_files/figure-gfm/unnamed-chunk-22-1.png)<!-- --> 
 
#### Combine the different heatmaps in one overview figure 
 


In [ ]:
figures_without_legend = plot_grid( 
  p_ligand_pearson + theme(legend.position = "none", axis.ticks = element_blank()) + theme(axis.title.x = element_text()), 
  p_ligand_tumor_expression + theme(legend.position = "none", axis.ticks = element_blank()) + theme(axis.title.x = element_text()) + ylab(""), 
  p_ligand_target_network + theme(legend.position = "none", axis.ticks = element_blank()) + ylab(""),  
  NULL, 
  NULL, 
  p_target_tumor_scaled_expression + theme(legend.position = "none", axis.ticks = element_blank()) + xlab(""),  
  align = "hv", 
  nrow = 2, 
  rel_widths = c(ncol(vis_ligand_pearson)+ 4.5, ncol(vis_ligand_tumor_expression), ncol(vis_ligand_target)) -2, 
  rel_heights = c(nrow(vis_ligand_pearson), nrow(vis_target_tumor_expression_scaled) + 3))  
 
legends = plot_grid( 
  as_ggplot(get_legend(p_ligand_pearson)), 
  as_ggplot(get_legend(p_ligand_tumor_expression)), 
  as_ggplot(get_legend(p_ligand_target_network)), 
  as_ggplot(get_legend(p_target_tumor_scaled_expression)), 
  nrow = 2, 
  align = "h") 
 
plot_grid(figures_without_legend,  
          legends,  
          rel_heights = c(10,2), nrow = 2, align = "hv") 


 
![](ligand_activity_geneset_files/figure-gfm/unnamed-chunk-23-1.png)<!-- --> 
 
## Other follow-up analyses: 
 
As another follow-up analysis, you can infer possible signaling paths 
between ligands and targets of interest. You can read how to do this in 
the following vignette [Inferring ligand-to-target signaling 
paths](ligand_target_signaling_path.md):`vignette("ligand_target_signaling_path", 
package="nichenetr")`. 
 
Another follow-up analysis is getting a “tangible” measure of how well 
top-ranked ligands predict the gene set of interest and assess which 
genes of the gene set can be predicted well. You can read how to do this 
in the following vignette [Assess how well top-ranked ligands can 
predict a gene set of 
interest](target_prediction_evaluation_geneset.md):`vignette("target_prediction_evaluation_geneset", 
package="nichenetr")`. 
 
In case you want to visualize ligand-target links between multiple 
interacting cells, you can make an appealing circos plot as shown in 
vignette [Circos plot visualization to show active ligand-target links 
between interacting cells](circos.md):`vignette("circos", 
package="nichenetr")`. 
 
## References 
 
<div id="refs" class="references"> 
 
<div id="ref-puram_single-cell_2017"> 
 
Puram, Sidharth V., Itay Tirosh, Anuraag S. Parikh, Anoop P. Patel, 
Keren Yizhak, Shawn Gillespie, Christopher Rodman, et al. 2017. 
“Single-Cell Transcriptomic Analysis of Primary and Metastatic Tumor 
Ecosystems in Head and Neck Cancer.” *Cell* 171 (7): 1611–1624.e24. 
<https://doi.org/10.1016/j.cell.2017.10.044>. 
 
</div> 
 
</div> 
